# Environment setup

In [1]:
from openai import OpenAI
import os
import re
import pandas as pd
from dotenv import load_dotenv
import requests

In [2]:
import subprocess
import base64
from IPython.display import display, Image
import json
import math

In [3]:
# Parent directory of the current directory
path = os.path.dirname(os.getcwd()) 

# Paths to the folders of example images and transcriptions
image_folder = path+'/data/Archives_LLN_Nivelles_I_1921_REG 5193'
text_folder = path+'/data/transcriptions'

In [4]:
load_dotenv()

True

In [5]:
openai_API_KEY = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(api_key=openai_API_KEY)

In [8]:
import mlflow 
mlflow.login()
# host: https://community.cloud.databricks.com
# id: [your email]

2024/08/26 10:18:12 INFO mlflow.utils.credentials: No valid Databricks credentials found, please enter your credentials...


Databricks Host (should begin with https://):  https://community.cloud.databricks.com
Username:  seorin.kim@vub.be
Password:  ········


2024/08/26 10:18:23 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


In [9]:
mlflow.set_tracking_uri("databricks")

In [10]:
EXPERIMENT_NAME = "/Users/seorin.kim@vub.be/transcripter_llm"
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2672828317980737', creation_time=1723544700863, experiment_id='2672828317980737', last_update_time=1724156024259, lifecycle_stage='active', name='/Users/seorin.kim@vub.be/transcripter_llm', tags={'mlflow.experiment.sourceName': '/Users/seorin.kim@vub.be/transcripter_llm',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'seorin.kim@vub.be',
 'mlflow.ownerId': '4512664544609245',
 'mlflow.sharedViewState.99df0ff2fdaf166603bef89139097b1348ce64798f7b39e42d272c4d9ee74228': '{"searchFilter":"","orderByKey":"attributes.start_time","orderByAsc":false,"startTime":"ALL","lifecycleFilter":"Active","datasetsFilter":[],"modelVersionFilter":"All '
                                                                                            'Runs","selectedColumns":["attributes.`Source`","attributes.`Models`","attributes.`Dataset`"],"runsExpanded":{},"runsPinned":[],"runsHidden":[],"runsHiddenMode":"FIRST_10_RUNS","vi

# Few shot trials (Doesn't work well -- but don't think it is relevant.)

In [6]:
def handle_nan(obj):
    return {k: (None if (isinstance(v, float) and math.isnan(v)) else v) for k, v in obj.items()}


# Example Transcription Data
def read_json_files(folder_path):
    json_data_list = []

    # Process each JSON file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)
            
            # Read JSON data from file
            with open(file_path, 'r', encoding='utf-8') as json_file:
                json_data_pre = json.load(json_file, object_hook=handle_nan)
                json_data =json.dumps(json_data_pre, indent=4, ensure_ascii=False)
                json_data_list.append(json_data)

    return json_data_list

# Folder containing the JSON files
json_folder = os.path.join(path, 'data/json_transcriptions')

# Read all JSON files from the folder
all_json_data = read_json_files(json_folder)

# Print out the loaded JSON data (for demonstration purposes)
for idx, json_data in enumerate(all_json_data):
    print(f"Data from JSON file {idx + 1}:")
    print(json_data)
    print("\n")

Data from JSON file 1:
[
    {
        "N' d'ordre": {
            "Unnamed: 0_level_1": null
        },
        "Date du dépot des déclarations": {
            "Unnamed: 1_level_1": 1921
        },
        "Désignation des personnes décédées ou absentes.": {
            "Nom.": null,
            "Prénoms": null,
            "Domiciles": null,
            "Domiciles.1": null
        },
        "Date du décès ous du judgement d'envoi en possession, en cas d'absence.": {
            "Unnamed: 6_level_1": null
        },
        "Noms, Prénoms et demeures des parties déclarantes.": {
            "Unnamed: 7_level_1": null
        },
        "Droits de succession en ligne collatérale et de mutation en ligne directe.": {
            "Actif. (2)": null,
            "Passif. (2)": null,
            "Restant NET. (2)": null
        },
        "Droit de mutation par déces": {
            "Valeur des immeubles. (2)": null
        },
        "Numéros des déclarations": {
            "Primitives."

In [13]:
# Example Image Data
def read_image_files(folder_path):
    example_paths = []

# Define the regex pattern
    pattern = re.compile(r'^example\d+.jpeg$')
    
    # Process each image file in the folder
    for file_name in os.listdir(folder_path):
        if pattern.match(file_name):
            file_path = os.path.join(folder_path, file_name)
            example_paths.append(file_path)

    return example_paths


In [15]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [17]:
all_images = read_image_files(image_folder)
all_images_encoded = [encode_image for image in all_images]

In [32]:
import requests

def create_example(prompt_example, encoded_img_ex, transcription_ex):
    return [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": prompt_example
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_img_ex}"
                    }
                },
                {
                    "type": "text",
                    "text": transcription_ex
                }
            ]
        }
    ]

In [33]:
examples = []

prompt_example = "You are a helpful assistant, recreate the table from this handwritten document into a json file, this table contains columns and subcolumns. No PROFESSION column. Copy the texts as they are, do not add any other sentences from you:"

for i in range(len(all_images_encoded)):
    example = create_example(prompt_example, all_images_encoded[i], all_json_data[i])
    examples.append(example)

In [34]:
new_image_encoded = encode_image(image_folder+'/IMG_2024_03_19_11_46_52_911.jpeg')
prompt = "You are a helpful assistant, recreate the table from this handwritten document into a json file, this table contain columns and subcolums. No PROFESSION column. Copy the texts as they are, do not add any other sentences from you:"

In [35]:
User_input = {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{new_image_encoded}"
                    }
                }
            ]
        }

Too many examples? Errors occur with the examples... Should look for a new way? \
Just do with `User_input` as messages and then iterate? 

In [36]:
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_API_KEY}"
}

payload = {
    "model": "gpt-4o",
    "messages": examples + [User_input],
    "max_tokens": 1000,
    "temperature": 0
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
print(response.json())


{'error': {'message': "Invalid type for 'messages[0]': expected an object, but got an array instead.", 'type': 'invalid_request_error', 'param': 'messages[0]', 'code': 'invalid_type'}}


# With Examples

In [11]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [20]:
def read_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as json_file:
        json_data_pre = json.load(json_file)
        json_data =json.dumps(json_data_pre, indent=4, ensure_ascii=False)
    return json_data

## Give already an example in the beginning

In [12]:
example_image = encode_image(path+'/data/Archives_LLN_Nivelles_I_1921_REG 5193/example2_cropped.jpeg')
input_image = encode_image(path+'/data/Archives_LLN_Nivelles_I_1921_REG 5193/example1_cropped.jpeg')
# transcription_json = read_json(os.path.join(path, 'data/json_transcriptions/transcription_ex2.json'))

In [13]:
ex2 = open(path+'/data/transcriptions/transcription_ex2.txt', 'r', encoding='utf-8').read()
ex2

'N\' d\'ordre\tDate du dépot des déclarations\tDésignation des personnes décédées ou absentes.\t\t\tDate du décès ou du judgement d\'envoi en possession, en cas d\'absence.\tNoms, Prénoms et demeures des parties déclarantes.\tDroits de succession en ligne collatérale et de mutation en ligne directe.\t\t\tDroit de mutation par déces\tNuméros des déclarations\t\tDate\t\tNuméros de la consignation des droits au sommier n\' 28\tRecette des droits et amendes.\t\tCautionnements. \tObservations\n\t\tNom.\tPrénoms\tDomiciles\t\t\tActif. (2)\tPassif. (2)\tRestant NET. (2)\tValeur des immeubles. (2)\tPrimitives.\tSupplémentaires.\tde l\'expiration du délai de rectification.\tde l\'exigibilité des droits.\t\tDate\tN^03\tNuméros de la consignation au sommier n\'30\t(les déclarations qui figurent à l\'état n\'413 doivent être émargées en conséquence, dans la présnete colonne.)\n\t1919\t\t\t\t\t\t\t\t\t\t506\t\t\t\t\t\t\t\t\n403\tquatre 9bre\tPayot\tAntoinette\tBraine l\'Alleud\t14 mai 1919\tPayot M

In [49]:
prompt = f"""
        From the example, you learned the handwriting of this Belgian record. You learned which alphabet and which number is written in which way.
        With this knowledge, now consider the following image to recreate:
        
        First, you read a two-level header in the table, which you recognize the same as the example as follows in the form of ("first level", "second level"):
        ```
            [("N' d'ordre", " "),
            ("Date du dépot des déclarations", " "),
            ("Désignation des personnes décédées ou absentes.:", "Nom."),
            ("Désignation des personnes décédées ou absentes.:", "Prénoms"),
            ("Désignation des personnes décédées ou absentes.:", "Domiciles"), 
            ("Date du décès ou du judgement d'envoi en possession, en cas d'absence.", " "),
            ("Noms, Prénoms et demeures des parties déclarantes.", " "),
            ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Actif. (2)"),
            ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Passif. (2)"),
            ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Restant NET. (2)"),
            ("Droit de mutation par déces", "Valeur des immeubles. (2)"), 
            ("Numéros des déclarations", "Primitives."),
            ("Numéros des déclarations", "Supplémentaires."), 
            ("Date", "de l'expiration du délai de rectification."),
            ("Date", "de l'exigibilité des droits."),
            ("Numéros de la consignation des droits au sommier n' 28", " "),
            ("Recette des droits et amendes.", "Date"),
            ("Recette des droits et amendes.", "N^03"),
            ("Cautionnements. ", "Numéros de la consignation au sommier n'30"),
            ("Observations (les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présnete colonne.)", " ")] 
         ```

        Context: 
        - It's written in French language and the names of the people are domiciles are Belgian.
        - Each row contains information about a dead person for the 20 variables above. Some rows contain information about the service date of the dead person written in the previous row. Such rows begin with texts like "Arrêté le \d{2} \w+ \d{4}( \w+)? servais" under "Nom." variable. 
        - When you see "Arrêté le \d{2} \w+ \d{4}( \w+)? servais", the subsequent row will be the next serviced day.
        - N' d'ordre will also follow an order. 
        - The family name in this column "Noms, Prénoms et demeures des parties déclarantes." may be the same as the family name in "Nom." column.
        
        Task: 
        Please recreate the table by filling in all the information in the record. Pay attention to reading each word and number correctly. 
            ```plaintext
"""

In [14]:
prompt = """
        Recreate the table you see in the image. At the start of each row of the table, can you add #?
        ```plaintext
"""

In [15]:

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_API_KEY}"
}

payload = {
    "model": "gpt-4o",
    "messages":  [
                {
                    "role": "system", 
                    "content": "You are a helpful assistant who can read old handwriting with a background in history, and you are going to recreate a scanned déclaration de succession from Belgium in a txt format."
                },
                {
                    "role": "user",
                    "content": [ 
                    {
                        "type": "text",
                        "text": f"""
                        The ```plaintext block is the example transcription of the example image you saw:

                        Transcription:
                        ```plaintext
                        {ex2}
                        ```
                        Compare what you read initially and the solution key in ```plaintext block. Learn how each letter and digit is written in the document. 

                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{example_image}", "detail": 'high'
                        }
                    },
                    ]
                },

                {
                    "role": "user",
                    "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{input_image}", 'detail': 'high'
                        }
                    }
                    ]
                }

                ],
    "max_tokens": 3000,
    "temperature": 0
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [16]:
content = response.json()["choices"][0]["message"]["content"]
print(content)

```plaintext
# N' d'ordre	Date du dépot des déclarations	Désignation des personnes décédées ou absentes.			Date du décès ou du jugement d'envoi en possession, en cas d'absence.	Noms, Prénoms et demeures des parties déclarantes.	Droits de succession en ligne collatérale et de mutation en ligne directe.			Droit de mutation par décès	Numéros des déclarations		Date		Numéros de la consignation des droits au sommier n' 28	Recette des droits et amendes.		Cautionnements. 	Observations
#	Nom.	Prénoms	Domiciles			Actif. (2)	Passif. (2)	Restant NET. (2)	Valeur des immeubles. (2)	Primitives.	Supplémentaires.	de l'expiration du délai de rectification.	de l'exigibilité des droits.		Date	N^03	Numéros de la consignation au sommier n'30	(les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présente colonne.)
# 1919										506								
# 398	vingt 8bre	Horremt		Stephaneff		19 9b 1919	Horremt Henri & autres	2250	2045	205		241/1919		15 8bre 1919	15 janvier 1920		non 

In [53]:
# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("model", payload["model"])
    mlflow.log_param("temperature", payload.get("temperature", 0))

    # Log the example prompt
    example_prompt = f"""
    System prompt:
    {payload['messages'][0]['content']}

    User example prompt:
    {payload['messages'][1]['content'][0]['text']}
    
    Example image base64:
    {example_image[:100]}... (truncated for logging)

    """
    mlflow.log_text(example_prompt, "example_prompt.txt")

    # Log the actual prompt
    actual_prompt = f"""
    User actual prompt:
    {prompt}

    Input image base64:
    {input_image[:100]}... (truncated for logging)
    """
    mlflow.log_text(actual_prompt, "actual_prompt.txt")

    # Make the API call
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Log the API response
    response_json = response.json()
    mlflow.log_text(json.dumps(response_json, indent=2), "response.json")

    # Extract and log relevant response information
    if "choices" in response_json:
        response_text = response_json["choices"][0]["message"]["content"]
        mlflow.log_text(response_text, "response_text.txt")
        mlflow.log_metric("response_length", len(response_text))

    # Log custom metrics or additional outputs as needed
    # Example: Log any evaluation metrics of the transcription quality

# End of the MLflow run

2024/08/20 14:14:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run brawny-gnu-416 at: https://community.cloud.databricks.com/ml/experiments/2672828317980737/runs/5ca22a6076864e3fa750d6c7b1657b00.
2024/08/20 14:14:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://community.cloud.databricks.com/ml/experiments/2672828317980737.


In [57]:
print(response_text)

```plaintext
N' d'ordre	Date du dépot des déclarations	Désignation des personnes décédées ou absentes.			Date du décès ou du judgement d'envoi en possession, en cas d'absence.	Noms, Prénoms et demeures des parties déclarantes.	Droits de succession en ligne collatérale et de mutation en ligne directe.			Droit de mutation par déces	Numéros des déclarations		Date		Numéros de la consignation des droits au sommier n' 28	Recette des droits et amendes.		Cautionnements. 	Observations
		Nom.	Prénoms	Domiciles			Actif. (2)	Passif. (2)	Restant NET. (2)	Valeur des immeubles. (2)	Primitives.	Supplémentaires.	de l'expiration du délai de rectification.	de l'exigibilité des droits.		Date	N^03	Numéros de la consignation au sommier n'30	(les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présnete colonne.)
1919										506								
398	vingt 8bre	Horcent	Stephane	Enghien	5 9bre 1919	Horcent Henri & autres	2250	2045	205		241/1919		15 8bre 1919	25 mars 1920		non pas

In [59]:

prompt_refine = f"""
        
        Your first draft:
        ```plaintext
        {response_text}
        ```

        Errors: 
        Your first transcription you made in ```plaintext block contains some errors.
        
        Task:
        Refine your first trasncription in ```plaintext block. 
        Make sure to read the names of the people and the location as well as the dates and the numbers correctly.
        Transcribe as you see in the image.
        ```plaintext
        """


headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai_API_KEY}"
}

payload = {
    "model": "gpt-4o",
    "messages":  [
                {
                    "role": "system", 
                    "content": "You are a helpful assistant who can read old handwriting with a background in history, and you are going to recreate a scanned déclaration de succession from Belgium in a txt format."
                },
                {
                    "role": "user",
                    "content": [ 
                    {
                        "type": "text",
                        "text": f"""
                        The ```plaintext block is the example transcription of the example image you saw:

                        Transcription:
                        ```plaintext
                        {ex2}
                        ```
                        Compare what you read initially and the solution key in ```plaintext block. Learn how each letter and digit is written in the document. 

                        """
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{example_image}", "detail": 'high'
                        }
                    },
                    ]
                },

                {
                    "role": "user",
                    "content": [
                    {
                        "type": "text",
                        "text": prompt_refine
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{input_image}", 'detail': 'high'
                        }
                    }
                    ]
                }

                ],
    "max_tokens": 3000,
    "temperature": 0
}
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

In [60]:
content = response.json()["choices"][0]["message"]["content"]
print(content)

```plaintext
N' d'ordre	Date du dépot des déclarations	Désignation des personnes décédées ou absentes.			Date du décès ou du jugement d'envoi en possession, en cas d'absence.	Noms, Prénoms et demeures des parties déclarantes.	Droits de succession en ligne collatérale et de mutation en ligne directe.			Droit de mutation par décès	Numéros des déclarations		Date		Numéros de la consignation des droits au sommier n' 28	Recette des droits et amendes.		Cautionnements. 	Observations
		Nom.	Prénoms	Domiciles			Actif. (2)	Passif. (2)	Restant NET. (2)	Valeur des immeubles. (2)	Primitives.	Supplémentaires.	de l'expiration du délai de rectification.	de l'exigibilité des droits.		Date	N^03	Numéros de la consignation au sommier n'30	(les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présente colonne.)
1919										506								
398	vingt 8bre	Horcent	Stephane	Enghien	5 9bre 1919	Horcent Henri & autres	2250	2045	205		241/1919		15 8bre 1919	25 mars 1920		non pass

## Separately

In [25]:
def call(prompt, max_tokens =1000, base64_image=None):
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {openai_API_KEY}"
        } 
        if base64_image:  
            payload = {
                "model": "gpt-4o",
                "messages": [
                {
                    "role": "system", 
                    "content": "You are a helpful assistant who can read old handwriting."},
                {
                    "role": "user",
                    "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                        "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                    ]
                }
                ],
                "max_tokens": max_tokens,
                "temperature": 0,
                # "logit_bias": {"1734": -100},
                # "response_format": { "type": "json_object" }
            }
        else:
            payload = {
                "model": "gpt-4o",
                "messages": [
                {
                    "role": "user",
                    "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                    ]
                }
                ],
                "max_tokens": max_tokens,
                "temperature": 0,
                # "logit_bias": {"1734": -100},
                # "response_format":{ "type": "json_object" }
            }
        
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        # return response.json()["choices"][0]["message"]["content"]
        return response


In [157]:
def makeDraft(image_path, output_format = "json"):
    base64_image = encode_image(image_path)
    
    prompt = f"""
        Structure:
                "N' d'ordre":
                ,
                "Date du dépot des déclarations": 
                ,
                "Désignation des personnes décédées ou absentes.": 
                    "Nom.":  ,
                    "Prénoms":  ,
                    "Domiciles": 
                ,
                "Date du décès ous du judgement d'envoi en possession, en cas d'absence.": 
                ,
                "Noms, Prénoms et demeures des parties déclarantes.": 
                ,
                "Droits de succession en ligne collatérale et de mutation en ligne directe.": 
                    "Actif. (2)": ,
                    "Passif. (2)": ,
                    "Restant NET. (2)": 
                ,
                "Droit de mutation par déces": 
                    "Valeur des immeubles. (2)": 
                ,
                "Numéros des déclarations": 
                    "Primitives.": ,
                    "Supplémentaires.": 
                ,
                "Date": 
                    "de l'expiration du délai de rectification.": ,
                    "de l'exigibilité des droits.": 
                ,
                "Numéros de la consignation des droits au sommier n' 28": 
                ,
                "Recette des droits et amendes.": 
                    "Date": ,
                    "N^03": 
                ,
                "Cautionnements. ": 
                    "Numéros de la consignation au sommier n'30":
                ,
                "Observations (les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présnete colonne.)": 
 
        
        Notations:
            7bre or 7b == September
            8bre or 8b == October
            9bre or 9b == November
            Dbre or Db == December
            d or " (quotation mark) = ditto    
        
        Tips: 
        The table in the image is a Déclaration de succession of Belgium. 
        Each deceased person should have information in the following structure below.
        Some notations for dates and others are used as indicated in Notations, but return the notations as they are seen, not what they mean.
        Information for 'Actif.', 'Passif.' and 'Restant Net.' should exist for each dead person. So, read them well from the image.
        'Restant Net.' is the result of 'Actif.' minus 'Passif.'.
        
        Task: 
        Please recreate the table in the image as a {output_format} file based on this structure.
        When you see Arrêté le \d{2} \w+ \d{4}( \w+)? servais, add it to a new key key called 'Note'. 
        If there is no information of the deceased name but only 'Arrêté le \d{2} \w+ \d{4}( \w+)? servais', add it under an empty name.
        Make sure to read the names of the people and the location as well as the dates and the numbers correctly.
        Do not make up information. 
    """
    return call(prompt, max_tokens=3000, base64_image = base64_image), base64_image

In [27]:
def makeDraft(image_path, output_format, example):
    base64_image = encode_image(image_path)
    
    prompt = f"""
           
            Header:
            ```
                [("N' d'ordre", " "),
                ("Date du dépot des déclarations", " "),
                ("Désignation des personnes décédées ou absentes.:", "Nom."),
                ("Désignation des personnes décédées ou absentes.:", "Prénoms"),
                ("Désignation des personnes décédées ou absentes.:", "Domiciles"), 
                ("Date du décès ou du judgement d'envoi en possession, en cas d'absence.", " "),
                ("Noms, Prénoms et demeures des parties déclarantes.", " "),
                ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Actif. (2)"),
                ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Passif. (2)"),
                ("Droits de succession en ligne collatérale et de mutation en ligne directe.", "Restant NET. (2)"),
                ("Droit de mutation par déces", "Valeur des immeubles. (2)"), 
                ("Numéros des déclarations", "Primitives."),
                ("Numéros des déclarations", "Supplémentaires."), 
                ("Date", "de l'expiration du délai de rectification."),
                ("Date", "de l'exigibilité des droits."),
                ("Numéros de la consignation des droits au sommier n' 28", " "),
                ("Recette des droits et amendes.", "Date"),
                ("Recette des droits et amendes.", "N^03"),
                ("Cautionnements. ", "Numéros de la consignation au sommier n'30"),
                ("Observations (les déclarations qui figurent à l'état n'413 doivent être émargées en conséquence, dans la présnete colonne.)", " ")] 
             ```
            
            Tips: 
            - The image contains a Déclaration de succession from Belgium. This implies that all the information written has to be Belgium-related.
            - Each row contains information about a dead person. Some row contains information about the service date of the dead person written in the previous row. Such rows contains texts like "Arrêté le \d{2} \w+ \d{4}( \w+)? servais". 
            - Consider the column explanations before filling in the table to decide whether you correctly read the information. 
            
            Task: 
                Consult the example and please recreate the table you read in the image in a {output_format} format. 
                Make sure to read the numbers and dates correctly.
    """
    return call(prompt, max_tokens=3000, base64_image = base64_image), base64_image

In [29]:
draft, encoded_image = makeDraft(image_path = path+'/data/Archives_LLN_Nivelles_I_1921_REG 5193/example1.jpeg', output_format = 'txt', example=ex2)

In [31]:
draft.json()

{'id': 'chatcmpl-9pApHNb4IxBrOgFHM9UHLwDMEHWc1',
 'object': 'chat.completion',
 'created': 1721983371,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "Here is the recreated table in text format based on the image provided:\n\n```\nN' d'ordre | Date du dépot des déclarations | Désignation des personnes décédées ou absentes.: Nom. | Désignation des personnes décédées ou absentes.: Prénoms | Désignation des personnes décédées ou absentes.: Domiciles | Date du décès ou du judgement d'envoi en possession, en cas d'absence. | Noms, Prénoms et demeures des parties déclarantes. | Droits de succession en ligne collatérale et de mutation en ligne directe.: Actif. (2) | Droits de succession en ligne collatérale et de mutation en ligne directe.: Passif. (2) | Droits de succession en ligne collatérale et de mutation en ligne directe.: Restant NET. (2) | Droit de mutation par déces: Valeur des immeubles. (2) | Numéros des déclarations: Pri

In [234]:
draft_json = draft.json()["choices"][0]["message"]["content"]

In [235]:
print(draft_json)

```txt
| DATE DE DÉCÈS | NOM, PRÉNOM | DOMICILE | PROFESSION | DATE DE L'ACTE | ACTIF | PASSIF | RESTANT | OBSERVATIONS |
|---------------|-------------|----------|-------------|----------------|-------|--------|---------|--------------|
| 6 1937 6 août | Hoornaert   |          |             |                |       |        |         |              |
|               | Lefevere    | Jules    |             |                |       |        |         |              |
|               |             |          |             |                |       |        |         |              |
|               |             |          |             |                |       |        |         |              |
| 6 1937 6 août | Desmedt     | Jeanne  |              |                |       |        |         |              |
| 6 1937 6 août | Monseur     |          |             |                |       |        |         |              |
| 6 1937 6 août | Bouly       |          |             |          

In [126]:
 def refineLayout(draft, json_path):
        
    all_json_data = read_json(json_path)   
 
    prompt = f"""
        
        Your first draft:
        ```draft
        {draft}
        ```
        
        Example:
        ```json
        {all_json_data}
        ```  

        Errors: 
        Your first draft in the ```draft block contains some errors. 
        
        Context: 
        The content of your draft in ```draft block should follow the structure in the example in the ```json block. 
        Information for 'Actif.', 'Passif.' and 'Restant Net.' should exist for each dead person. So, read them well from the image.
        'Restant Net.' is 'Actif.' - 'Passif.'
        
        Task:
        Refine your first draft based on the example and the image.
        When you see Arrêté le \d{2} \w+ \d{4}( \w+)? servais, add it to a new key key called 'Note'. 
        If there is no information of the deceased name but only 'Arrêté le \d{2} \w+ \d{4}( \w+)? servais', add it under an empty name.
        Make sure to read the names of the people and the location as well as the dates and the numbers correctly.
        Do not make up information. 
        
        """
    return call(prompt, max_tokens = 3000, base64_image = encoded_image)

In [128]:
refine = refineLayout(draft_json, json_path = os.path.join(path, 'data/json_transcriptions/transcription_ex2.json'))

In [129]:
refine_json = refine.json()["choices"][0]["message"]["content"]
print(refine_json)

       {
    "entries": [
        {
            "N' d'ordre": "6",
            "Date du dépot des déclarations": "1897 8bre",
            "Désignation des personnes décédées ou absentes": {
                "Nom": "",
                "Prénoms": "",
                "Domiciles": ""
            },
            "Date du décès ous du jugement d'envoi en possession, en cas d'absence": "",
            "Noms, Prénoms et demeures des parties déclarantes": "",
            "Droits de succession en ligne collatérale et de mutation en ligne directe": {
                "Actif. (2)": "",
                "Passif. (2)": "",
                "Restant NET. (2)": ""
            },
            "Droit de mutation par décès": {
                "Valeur des immeubles. (2)": ""
            },
            "Numéros des déclarations": {
                "Primitives.": "",
                "Supplémentaires.": ""
            },
            "Date": {
                "de l'expiration du délai de rectification": "",
       

In [70]:
def checkNames(content, website_name, base64_image):
    prompt = f"""
    Your first draft:
    {content}
    
    Website:
    {website_name}
    
    Task:
    There are some transcription errors in 'Nom', 'Prénoms', and 'Noms, Prénoms et demeures des parties déclarantes' in your first draft.
    Read these items from the image again such that the corresponding names exist in Belgium according to the website.
    When you see Arrêté le \d{2} \w+ \d{4}( \w+)? servais, add it to a new key key called 'Note'. 
    If there is no information of the deceased name but only 'Arrêté le \d{2} \w+ \d{4}( \w+)? servais', add it under an empty name.
    Make sure to read the names of the people and the location as well as the dates and the numbers correctly.
    Only update those items. 
    Do not make up information.
    
    Tips:
    The family name in 'Nom' under 'Désignation des personnes décédées ou absentes.' may equal to the family name in 'Noms, Prénoms et demeures des parties déclarantes', which contains the family and first name of the declaring parties.
    But it is most likely that their first names (Prénoms) are different. 
    If the family names in 'Nom' and 'Noms, Prénoms et demeures des parties déclarantes' are the same, 'Prénoms' should be masculine.
    'Noms, Prénoms et demeures des parties déclarantes' may end with '& autre' or '& autres'.
   
    """
    return call(prompt, max_tokens=3000, base64_image=base64_image)

In [72]:
checkNames = checkNames(refine_json, "https://nl.geneanet.org/genealogie/", encoded_image)

In [73]:
checkNames_json = checkNames.json()["choices"][0]["message"]["content"]

In [74]:
print(checkNames_json )


    {
    "entries": [
        {
            "N' d'ordre": "6",
            "Date du dépot des déclarations": "1897 8bre",
            "Désignation des personnes décédées ou absentes.": {
                "Nom.": "",
                "Prénoms": "",
                "Domiciles": ""
            },
            "Date du décès ous du jugement d'envoi en possession, en cas d'absence.": "Arrêté le 10 7bre 1897 servais",
            "Noms, Prénoms et demeures des parties déclarantes.": "",
            "Droits de succession en ligne collatérale et de mutation en ligne directe.": {
                "Actif. (2)": "",
                "Passif. (2)": "",
                "Restant NET. (2)": ""
            },
            "Droit de mutation par déces": {
                "Valeur des immeubles. (2)": ""
            },
            "Numéros des déclarations": {
                "Primitives.": "",
                "Supplémentaires.": ""
            },
            "Date": {
                "de l'expiration du dél

There are still some errors in 'Domiciles'.
    Check whether 'Domiciles' in your draft in the ```json block exist in Sector_{lang} of the province data in the ```csv_1 block where Commune_{lang} is {municipality}.
    If not, consider the sectors in the province data.
    
    If you do not find anything that resembles the texts in the image in the province data, you can write it as you see but with an asterisk.
    
    Update 'Domiciles' in your draft in the ```json block.
    Do not make up for missing information.
    
    Tips:
    The sector name written in 'Domiciles' in the image may have similar name but not exactly the same as the sector name in the province data in ```csv block. 

In [94]:
def checkCities(content, country, province, municipality, location_path, base64_image, language="French", lang="FR"):
    txt = pd.read_csv(location_path, sep='\t')
    province = txt[txt['Province'] == {province}].copy()

    prompt = f"""
    
    Your draft:
     ```json 
    {content}
    ```
    
    Province data:
    ```txt
    {province}
    ```
    
    Task:
    There may be errors in the information you filled in 'Domiciles' in your draft in ```json block. Refine it.
    To improve 'Domiciles' in your draft, consult Sector_{lang} in the province data in ```txt block and the image. 
    When you see Arrêté le \d{2} \w+ \d{4}( \w+)? servais, add it to a new key key called 'Note'. 
    If there is no information of the deceased name but only 'Arrêté le \d{2} \w+ \d{4}( \w+)? servais', add it under an empty name.
    Make sure to read the names of the people and the location as well as the dates and the numbers correctly.
    
    Tips:
    The sector names in your draft and the province data may slightly differ.
    'Domiciles' in your draft should contain sector names in Sector_{lang}.

    
    """
    return call(prompt, max_tokens=3000, base64_image=base64_image)

In [96]:
location_path = os.path.join(path,'data_rag/BE_location_full.txt')

checkCities = checkCities(content = checkNames, country = "Belgium", province = "Brabant wallon", 
                          municipality = "Nivelles", location_path = location_path, 
                          base64_image = encoded_image, 
                          language = "French", lang="FR")

In [97]:
checkCities_json = checkCities.json()["choices"][0]["message"]["content"]

In [98]:
print(checkCities_json)


    {
      "Response": "[200]",
      "Domiciles": [
        {
          "Name": "Leboeuf",
          "Date of Decree": "1897",
          "Sector_FR": "Herve",
          "Note": "Arrêté le vingt-neuf octobre 1897 servais"
        },
        {
          "Name": "Lefevre",
          "Date of Decree": "1897",
          "Sector_FR": "Herve",
          "Note": "Arrêté le vingt-neuf octobre 1897 servais"
        },
        {
          "Name": "Desmet",
          "Date of Decree": "1897",
          "Sector_FR": "Bruxelles",
          "Note": "Arrêté le deux novembre 1897 servais"
        },
        {
          "Name": "Monseu",
          "Date of Decree": "1897",
          "Sector_FR": "Bruxelles",
          "Note": "Arrêté le deux novembre 1897 servais"
        },
        {
          "Name": "Bouly",
          "Date of Decree": "1897",
          "Sector_FR": "Bruxelles",
          "Note": "Arrêté le deux novembre 1897 servais"
        },
        {
          "Name": "Godart",
          "Dat

😢 Having troubles with the domiciles & numbers

# With Claude

In [ ]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"